[Source](https://towardsdatascience.com/how-to-build-a-simple-kafka-producer-and-consumer-with-python-a967769c4742)

In [14]:
from confluent_kafka import Producer
from faker import Faker
import random
import json
import time
import logging

faker = Faker()

In [15]:
logging.basicConfig(format='%(asctime)s %(message)s',
                    datefmt='%Y-%m-%d %H:%M:%S',
                    filename='producer.log',
                    filemode='w')

logger = logging.getLogger()
logger.setLevel(logging.INFO)

## Build Kafka Producer

In [16]:
# create the producer by specifying the port of your Kafka cluster

producer = Producer({'bootstrap.servers': '192.168.24.254:9092'})
print('Kafka Producer has been initiated...')

Kafka Producer has been initiated...


In [17]:
# Define a callback function that takes care of acknowledging new messages or errors.
def receipt(err,msg):
    if err is not None:
        print('Error: {}'.format(err))
    else:
        message = 'Produced message on topic {} with value of {} \n'.format(msg.topic(), msg.value().decode('utf-8'))
        logger.info(message)
        print(message)

In [21]:
for i in range(10):
    data={
        'user_id': faker.random_int(min=20000, max=100000),
        'user_name':faker.name(),
        'user_address':faker.street_address() + ' | ' + faker.city() + ' | ' + faker.country_code(),
        'platform': random.choice(['Mobile', 'Laptop', 'Tablet']),
        'signup_at': str(faker.date_time_this_month())    
    }
    m=json.dumps(data)
    producer.poll(1)
    producer.produce('user-tracker', m.encode('utf-8'),callback=receipt)
    producer.flush()
    time.sleep(3)

Produced message on topic user-tracker with value of {"user_id": 74844, "user_name": "Mr. Jeffrey Mullins", "user_address": "8338 Christine Motorway Apt. 094 | East Brittneyville | KI", "platform": "Mobile", "signup_at": "2023-07-16 13:37:34"}

Produced message on topic user-tracker with value of {"user_id": 36918, "user_name": "Alejandra Murray", "user_address": "626 Perry Meadow Suite 585 | Kyleborough | UZ", "platform": "Tablet", "signup_at": "2023-07-20 04:09:22"}

Produced message on topic user-tracker with value of {"user_id": 65544, "user_name": "Rebecca Blanchard", "user_address": "7993 Jones Rapid | Tiffanyfurt | TH", "platform": "Tablet", "signup_at": "2023-07-14 05:49:56"}

Produced message on topic user-tracker with value of {"user_id": 50172, "user_name": "Brian Mcneil", "user_address": "2827 Robertson Roads Suite 873 | West Allisonside | PS", "platform": "Laptop", "signup_at": "2023-07-03 02:45:41"}

Produced message on topic user-tracker with value of {"user_id": 64138, 

## Build Kafka Consumer

In [22]:
from confluent_kafka import Consumer
################
consumer=Consumer({'bootstrap.servers':'192.168.24.254:9092','group.id':'python-consumer','auto.offset.reset':'earliest'})
print('Kafka Consumer has been initiated...')

Kafka Consumer has been initiated...


In [24]:
print('Available topics to consume: ', consumer.list_topics().topics)
consumer.subscribe(['user-tracker'])

Available topics to consume:  {'SomeTopic': TopicMetadata(SomeTopic, 1 partitions), 'test_topic': TopicMetadata(test_topic, 1 partitions), 'user-tracker': TopicMetadata(user-tracker, 1 partitions), '__consumer_offsets': TopicMetadata(__consumer_offsets, 50 partitions)}


In [ ]:
while True:
    msg=consumer.poll(1.0) #timeout
    if msg is None:
        continue
    if msg.error():
        print('Error: {}'.format(msg.error()))
        continue
    data=msg.value().decode('utf-8')
    print(data)
consumer.close()

In [28]:
consumer.close()

%4|1689828648.002|SESSTMOUT|rdkafka#consumer-2| [thrd:main]: Consumer group session timed out (in join-state steady) after 207415 ms without a successful response from the group coordinator (broker 1, last error was Success): revoking assignment and rejoining group
%5|1689828648.002|REQTMOUT|rdkafka#consumer-2| [thrd:192.168.24.254:9092/bootstrap]: 192.168.24.254:9092/1: Timed out FetchRequest in flight (after 206186ms, timeout #0)
%4|1689828648.002|REQTMOUT|rdkafka#consumer-2| [thrd:192.168.24.254:9092/bootstrap]: 192.168.24.254:9092/1: Timed out 1 in-flight, 0 retry-queued, 0 out-queue, 0 partially-sent requests
%3|1689828648.002|FAIL|rdkafka#consumer-2| [thrd:192.168.24.254:9092/bootstrap]: 192.168.24.254:9092/1: 1 request(s) timed out: disconnect (after 2445641ms in state UP)
%3|1689828649.638|FAIL|rdkafka#consumer-2| [thrd:192.168.24.254:9092/bootstrap]: 192.168.24.254:9092/1: Failed to connect to broker at 192.168.24.254:9092: Network is down (after 1635ms in state CONNECT)
%4|16